In [1]:
import pandas as pd
import torch

In [2]:
tonight_odds = pd.read_excel('games_odds.xlsx', sheet_name='Tonight')
prev_odds = pd.read_excel('games_odds.xlsx', sheet_name='Previous')

In [3]:
import os
os.chdir('..')
from team_functions import dataPrep, getTeamRollingSeason, mapTeamID
from models import *

In [4]:
games_to_pred = []
teams, opps = tonight_odds['visitor'], tonight_odds['local']
for team, opp in zip(teams, opps):
    team_row = {}
    team_df = pd.read_excel(f'data/CurrentSeason/{team}.xlsx')
    
    team_row['Game'] = team_df['Game'].iloc[-1] + 1
    team_row['Date'] = tonight_odds.loc[tonight_odds['visitor'] == team, 'date'].iloc[0]
    team_row['Streak'] = team_df['Streak'].iloc[-1]

    team_cols = ['Pts', 'Pace', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in team_cols:
        team_row[col] = team_df[col].mean()

    opp_df = pd.read_excel(f'data/CurrentSeason/{opp}.xlsx')
    team_row['OppID'] = mapTeamID(opp)
    
    opp_cols = ['Pts', 'eFG', 'TOV', 'ORB', 'FTR', 'ORT']
    for col in opp_cols:
        team_row['Opp'+col] = opp_df[col].mean()
        
    team_row['Location'] = 0
    
    team_row['Month'] = team_row['Date'].month
    team_row['DayOfWeek'] = team_row['Date'].weekday()
    team_row['DaysOfRest'] = (team_row['Date'] - team_df['Date'].iloc[-1]).days
    
    team_row['TeamID'] = mapTeamID(team)
    team_row['Season'] = 2024
    
    games_to_pred.append(team_row)
    
games_df = pd.DataFrame(games_to_pred)

In [5]:
games_df

,Game,Date,Streak,Pts,Pace,eFG,TOV,ORB,FTR,ORT,...,OppTOV,OppORB,OppFTR,OppORT,Location,Month,DayOfWeek,DaysOfRest,TeamID,Season
0,68,2024-03-21,1,118.044776,98.856716,0.561552,11.229851,23.819403,0.173328,118.074627,...,11.785507,19.936232,0.171928,111.184058,0,3,3,3,26,2024
1,69,2024-03-21,3,115.970588,97.461765,0.558926,11.354412,25.342647,0.206132,118.852941,...,12.746377,25.759420,0.222812,114.410145,0,3,3,2,19,2024
2,70,2024-03-21,-4,111.405797,96.471014,0.534159,10.966667,24.644928,0.178855,114.762319,...,10.864706,21.452941,0.221647,120.251471,0,3,3,2,3,2024
3,70,2024-03-21,2,112.028986,95.427536,0.533145,10.572464,24.524638,0.189072,115.515942,...,10.886765,24.741176,0.203853,114.610294,0,3,3,3,4,2024
4,69,2024-03-21,-2,117.191176,99.054412,0.544118,12.873529,28.686765,0.208809,117.470588,...,10.626087,22.213043,0.197710,119.142029,0,3,3,3,29,2024
5,69,2024-03-21,4,112.044118,94.857353,0.535529,11.201471,29.600000,0.196676,118.147059,...,10.747826,25.310145,0.174362,119.408696,0,3,3,3,20,2024
6,69,2024-03-21,-1,119.044118,99.702941,0.537691,10.838235,27.463235,0.209191,118.592647,...,12.583824,25.063235,0.231632,118.973529,0,3,3,3,1,2024


In [6]:
import joblib

In [7]:
def makeModelPredictions(games_df, odds_df, model_name='LR', nn_name='RNN',show=True):
    if model_name == 'NN':
        if nn_name == 'FCNN':
            fcnns = pd.read_excel('Models/TrainedModels/FCNNs/models.xlsx')
            fcnns = fcnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = fcnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        elif nn_name == 'RNN':
            rnns =  pd.read_excel('Models/TrainedModels/Recurrents/models.xlsx')
            rnns = rnns.sort_values(by='Accuracy', ascending=False, ignore_index=True)

            model_file = rnns.iloc[0]['File']

            model = torch.load('Models/'+model_file)
        
        scaler = joblib.load(f'Models/Scalers/{nn_name}.pkl')
        
    else:
        model = joblib.load(f'Models/TrainedModels/{model_name}_Model.pkl')
        scaler = joblib.load(f'Models/Scalers/{model_name}.bin')
        
    
    scaled_df = scaler.transform(games_df.drop(columns=['Date']))
    if model_name == 'NN':
        scaled_df = torch.FloatTensor(scaled_df)
        predictions = torch.round(model(scaled_df).squeeze(1))
        predictions = predictions.detach().numpy()
    else:
        predictions = model.predict(scaled_df)
    
    if model_name == 'NN':
        model_name = nn_name
        
    odds_df['Predictions'] = predictions
    if show:
        print(f'Predictions made by: {model_name}')
        display(odds_df)
        
    odds_df.to_excel(f'Predictions/ModelsPredictions/{model_name}.xlsx', index=0)

In [8]:
makeModelPredictions(games_df, tonight_odds, 'LR')

Predictions made by: LR


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-21,SAC,WAS,1.160000,5.550000,1
1,2024-03-21,NOP,ORL,1.675676,2.240000,1
2,2024-03-21,BRK,MIL,4.400000,1.227273,0
3,2024-03-21,CHI,HOU,2.420000,1.588235,1
4,2024-03-21,UTA,DAL,7.500000,1.100000,0
5,2024-03-21,NYK,DEN,4.200000,1.243902,0
6,2024-03-21,ATL,PHO,4.300000,1.235294,0


In [9]:
makeModelPredictions(games_df, tonight_odds, 'RF')

Predictions made by: RF


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-21,SAC,WAS,1.160000,5.550000,1
1,2024-03-21,NOP,ORL,1.675676,2.240000,0
2,2024-03-21,BRK,MIL,4.400000,1.227273,0
3,2024-03-21,CHI,HOU,2.420000,1.588235,1
4,2024-03-21,UTA,DAL,7.500000,1.100000,0
5,2024-03-21,NYK,DEN,4.200000,1.243902,0
6,2024-03-21,ATL,PHO,4.300000,1.235294,0


In [10]:
makeModelPredictions(games_df, tonight_odds, 'GB')

Predictions made by: GB


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-21,SAC,WAS,1.160000,5.550000,1
1,2024-03-21,NOP,ORL,1.675676,2.240000,0
2,2024-03-21,BRK,MIL,4.400000,1.227273,0
3,2024-03-21,CHI,HOU,2.420000,1.588235,0
4,2024-03-21,UTA,DAL,7.500000,1.100000,0
5,2024-03-21,NYK,DEN,4.200000,1.243902,0
6,2024-03-21,ATL,PHO,4.300000,1.235294,0


In [11]:
makeModelPredictions(games_df, tonight_odds, 'SVM')

Predictions made by: SVM


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-21,SAC,WAS,1.160000,5.550000,1
1,2024-03-21,NOP,ORL,1.675676,2.240000,1
2,2024-03-21,BRK,MIL,4.400000,1.227273,0
3,2024-03-21,CHI,HOU,2.420000,1.588235,0
4,2024-03-21,UTA,DAL,7.500000,1.100000,0
5,2024-03-21,NYK,DEN,4.200000,1.243902,0
6,2024-03-21,ATL,PHO,4.300000,1.235294,0


In [12]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'FCNN')

Predictions made by: FCNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-21,SAC,WAS,1.160000,5.550000,1.0
1,2024-03-21,NOP,ORL,1.675676,2.240000,1.0
2,2024-03-21,BRK,MIL,4.400000,1.227273,0.0
3,2024-03-21,CHI,HOU,2.420000,1.588235,1.0
4,2024-03-21,UTA,DAL,7.500000,1.100000,0.0
5,2024-03-21,NYK,DEN,4.200000,1.243902,0.0
6,2024-03-21,ATL,PHO,4.300000,1.235294,0.0


In [13]:
makeModelPredictions(games_df, tonight_odds, 'NN', 'RNN')

Predictions made by: RNN


,date,visitor,local,v_line,l_line,Predictions
0,2024-03-21,SAC,WAS,1.160000,5.550000,1.0
1,2024-03-21,NOP,ORL,1.675676,2.240000,1.0
2,2024-03-21,BRK,MIL,4.400000,1.227273,0.0
3,2024-03-21,CHI,HOU,2.420000,1.588235,1.0
4,2024-03-21,UTA,DAL,7.500000,1.100000,0.0
5,2024-03-21,NYK,DEN,4.200000,1.243902,0.0
6,2024-03-21,ATL,PHO,4.300000,1.235294,0.0
